# **Tasas de mortalidad**

## **Librerías y modulos necesarios**

In [6]:
import warnings
import itertools
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

## **Tasas crudas de mortalidad**

Se calcularon las tasas crudas de mortalidad por cada 100.000 personas, para cada departamento y por año, utilizando la siguiente fórmula:

$$
T_{kj} = \left( \frac{n_{kj}}{p_j} \right) \times k
$$

donde:

- $ T_{kj} $ es la tasa cruda de mortalidad para el año j y el departamento correspondiente.  
- $ n_{kj}$ es el número de muertes registradas.  
- $ p_j $ es la población total en ese año.  
- $ k $ es el factor de estandarización, en este caso $ k = 100.000$.


```{note}
Es importante tener en cuenta que el conjunto de datos utilizado solo registra muertes en mujeres mayores de 40 años, en el periodo comprendido entre los años 2009 y 2023. Por esta razón, las poblaciones utilizadas para el cálculo de las tasas corresponden exclusivamente a mujeres de 40 años o más, dentro de ese intervalo temporal.
```

In [7]:
url_muertes = 'https://github.com/sePerezAlbor/Data/blob/main/data_to_tasas.csv?raw=true'
url_poblacion = 'https://github.com/sePerezAlbor/Data/blob/main/POB-M40-DEP-2009-2023.xlsx?raw=true'

muertes = pd.read_csv(url_muertes).drop(columns=['Unnamed: 0'])
poblacion = pd.read_excel(url_poblacion)

equivalencias = {
    'SANTAFE DE BOGOTA D.C': 'BOGOTA DC',
    'BOGOTÁ, D.C.': 'BOGOTA DC',
    'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA': 'ARCHIPIÉLAGO DE SAN ANDRÉS',
    'BOLIVAR': 'BOLÍVAR',
    'BOYACA': 'BOYACÁ',
    'ATLANTICO': 'ATLÁNTICO',
    'CORDOBA': 'CÓRDOBA',
    'GUAINIA': 'GUAINÍA',
    'GUAVIARE': 'GUAVIARE',
    'VAUPES': 'VAUPÉS',
    'VICHADA': 'VICHADA',
    'NORTE DE SANTANDER': 'NORTE DE SANTANDER',
    'CHOCÓ': 'CHOCÓ'
}

muertes.rename(columns={'Nombre_Departamento_Def': 'DPNOM', 'año_def': 'ANIO'}, inplace=True)
muertes['DPNOM'] = muertes['DPNOM'].replace(equivalencias)
muertes['Nombre_Departamento_Res'] = muertes['Nombre_Departamento_Res'].replace(equivalencias)

poblacion['DPNOM'] = poblacion['DPNOM'].replace(equivalencias).str.upper()

cond = (muertes['DPNOM'] == 'BOGOTA DC') | (muertes['Nombre_Departamento_Res'] == 'BOGOTA DC')
muertes.loc[cond, ['DPNOM', 'Nombre_Departamento_Res']] = 'CUNDINAMARCA'

años = list(range(2009, 2024))
departamentos = muertes['DPNOM'].unique()
combinaciones = pd.DataFrame(list(itertools.product(departamentos, años)), columns=['DPNOM', 'ANIO'])

muertes_agg = muertes.groupby(['DPNOM', 'ANIO']).size().reset_index(name='FALLECIDAS')
muertes_completa = combinaciones.merge(muertes_agg, on=['DPNOM', 'ANIO'], how='left').fillna(0)
muertes_completa['FALLECIDAS'] = muertes_completa['FALLECIDAS'].astype(int)

muertes = muertes_completa.copy()

cols_mujeres = [col for col in poblacion.columns if col.startswith('Mujeres_')]
poblacion['TOTAL_MUJERES'] = poblacion[cols_mujeres].replace(',', '', regex=True).astype(float).sum(axis=1)

departamentos_amazona = ['AMAZONAS', 'GUAINÍA', 'GUAVIARE', 'VAUPÉS', 'VICHADA']

def agrupar_grupo(df, cols_sum, nombre_grupo):
    agrupado = df[df['DPNOM'].isin(departamentos_amazona)].groupby('ANIO', as_index=False).sum(numeric_only=True)
    agrupado['DPNOM'] = nombre_grupo
    cols = ['DPNOM', 'ANIO'] + [col for col in agrupado.columns if col not in ['DPNOM', 'ANIO']]
    return agrupado[cols]

amazona_poblacion = agrupar_grupo(poblacion, cols_mujeres + ['TOTAL_MUJERES'], 'GRUPO AMAZONA')
amazona_muertes = agrupar_grupo(muertes, ['FALLECIDAS'], 'GRUPO AMAZONA')

poblacion = pd.concat([poblacion[~poblacion['DPNOM'].isin(departamentos_amazona)], amazona_poblacion], ignore_index=True)
muertes = pd.concat([muertes[~muertes['DPNOM'].isin(departamentos_amazona)], amazona_muertes], ignore_index=True)

def calcular_tasa_mortalidad(muertes, poblacion, año_inicio=2009, año_fin=2023, k=100000):
    muertes_filtrado = muertes[(muertes['ANIO'] >= año_inicio) & (muertes['ANIO'] <= año_fin)]
    pob_filtrada = poblacion[(poblacion['ANIO'] >= año_inicio) & (poblacion['ANIO'] <= año_fin)]

    muertes_agg = muertes_filtrado.groupby('DPNOM', as_index=False)['FALLECIDAS'].sum()
    pob_agg = pob_filtrada.groupby('DPNOM', as_index=False)['TOTAL_MUJERES'].sum()

    tasa = pd.merge(muertes_agg, pob_agg, on='DPNOM')
    tasa['Tasa_Mortalidad'] = (tasa['FALLECIDAS'] / tasa['TOTAL_MUJERES']) * k 

    return tasa


In [8]:
tasa_mortalidad  = calcular_tasa_mortalidad(muertes, poblacion, año_inicio=2009, año_fin=2023, k = 100000)

In [9]:
tasa_mortalidad

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad
0,ANTIOQUIA,6185,19011651.0,32.532682
1,ARAUCA,137,543874.0,25.189658
2,ATLÁNTICO,3122,6773319.0,46.092617
3,BOLÍVAR,1588,5135499.0,30.922019
4,BOYACÁ,748,3751470.0,19.938851
5,CALDAS,1012,3393571.0,29.821094
6,CAQUETÁ,203,852787.0,23.804303
7,CASANARE,159,862417.0,18.436557
8,CAUCA,631,3713294.0,16.992999
9,CESAR,743,2578697.0,28.813001


In [10]:
mayor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmax()]
menor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmin()]
promedio = tasa_mortalidad['Tasa_Mortalidad'].mean()

print("\nMayor tasa:", mayor['DPNOM'], f"{mayor['Tasa_Mortalidad']:.2f}")
print("Menor tasa:", menor['DPNOM'], f"{menor['Tasa_Mortalidad']:.2f}")
print("Promedio:", f"{promedio:.2f}")


Mayor tasa: CUNDINAMARCA 121.61
Menor tasa: CHOCÓ 4.93
Promedio: 29.72


## **Tasas ajustadas por edad**

Para estimar tasas ajustadas de mortalidad por cáncer y poder realizar comparaciones válidas entre diferentes regiones o periodos de tiempo, se utilizó el método directo de ajuste por edad, tomando como referencia la **población estándar de Segi (1960)**. Esta población fue diseñada para facilitar la comparación internacional de tasas de mortalidad y morbilidad, y se basa en una distribución teórica de población que representa una media ponderada de estructuras poblacionales de distintas regiones del mundo en ese momento.  
El ajuste por edad corrige las posibles distorsiones generadas por diferencias en la estructura etaria de las poblaciones observadas, haciendo más precisa la comparación del riesgo de morir por una causa específica, como el cáncer, entre distintos departamentos o periodos.  
La tasa ajustada se expresa por cada **100.000 habitantes**, lo cual es un estándar internacional que facilita la comunicación y análisis de datos epidemiológicos.

Ahmad, O. B., Boschi-Pinto, C., Lopez, A. D., Murray, C. J., Lozano, R., & Inoue, M. (2001). Age standardization of rates: a new WHO standard. Geneva: World Health Organization. GPE Discussion Paper Series: No. 31.


| Grupo de Edad        | Intervalo                   | Proporción Segi (%) |
|----------------------|-----------------------------|----------------------|
| **Grupo_edad1**      | 40–49 años                  | 6.0 + 6.0 = **12.00** |
| **Grupo_edad2**      | 50–59 años                  | 5.0 + 4.0 = **9.00**  |
| **Grupo_edad3**      | 60–69 años                  | 4.0 + 3.0 = **7.00**  |
| **Grupo_edad4**      | 70 años en adelante (70+)   | 2.0 + 1.0 + 0.5 + 0.5 = **4.00** |



In [11]:
def procesar_tasas_por_periodo(año_inicio, año_fin):
    # Cargar datos de muertes
    url_muertes_complete = 'https://github.com/sePerezAlbor/Data/blob/main/data_to_tasas.csv?raw=true'
    muertes_df = pd.read_csv(url_muertes_complete)
    if 'Unnamed: 0' in muertes_df.columns:
        muertes_df.drop(columns=['Unnamed: 0'], inplace=True)

    condicion = (muertes_df['Nombre_Departamento_Def'] == 'BOGOTA DC') | (muertes_df['Nombre_Departamento_Res'] == 'BOGOTA DC')

    # Aplicar los cambios a las columnas DPNOM y Nombre_Departamento_Res para las filas que cumplen la condición
    muertes_df.loc[condicion, ['Nombre_Departamento_Def', 'Nombre_Departamento_Res']] = 'CUNDINAMARCA'

    muertes_df = muertes_df[(muertes_df['año_def'] >= año_inicio) & (muertes_df['año_def'] <= año_fin)]

    columnas_muertes = ['Nombre_Departamento_Def', 'grupo_edad']
    conteos_muertes = muertes_df.groupby(columnas_muertes, observed=False).size().reset_index(name='count')

    # Agrupar departamentos amazónicos
    departamentos_amazona = ['AMAZONAS', 'GUAINÍA', 'GUAVIARE', 'VAUPÉS', 'VICHADA']
    amazona_muertes_df = conteos_muertes[conteos_muertes['Nombre_Departamento_Def'].isin(departamentos_amazona)]
    
    if not amazona_muertes_df.empty:
        amazona_sum_df = amazona_muertes_df.groupby(['grupo_edad'], observed=False)['count'].sum().reset_index()
        amazona_sum_df['Nombre_Departamento_Def'] = 'GRUPO AMAZONA'
        conteos_muertes = conteos_muertes[~conteos_muertes['Nombre_Departamento_Def'].isin(departamentos_amazona)]
        conteos_muertes = pd.concat([conteos_muertes, amazona_sum_df[['Nombre_Departamento_Def', 'grupo_edad', 'count']]], ignore_index=True)

    grupos_edad_estandar = ['40-54 años', '55-64 años', '65-74 años', '75+ años']
    departamentos_unicos = conteos_muertes['Nombre_Departamento_Def'].unique()

    if len(departamentos_unicos) == 0: # No hay datos de muertes para el período
        cols_resumen = ['DPNOM', 'FALLECIDAS', 'TOTAL_MUJERES', 'Tasa_Mortalidad_Cruda', 'TAE']
        cols_detalle = ['DPNOM', 'grupo_edad', 'count', 'poblacion']
        return pd.DataFrame(columns=cols_resumen), pd.DataFrame(columns=cols_detalle)


    all_combinations = pd.DataFrame(list(itertools.product(departamentos_unicos, grupos_edad_estandar)), columns=['Nombre_Departamento_Def', 'grupo_edad'])
    
    # Si los 'grupo_edad' de 'conteos_muertes' no están en el formato estándar, el siguiente merge podría no ser efectivo.
    # Se recomienda pre-procesar 'grupo_edad' en 'muertes_df' para que coincida con 'grupos_edad_estandar'.
    muertes_completas_df = pd.merge(all_combinations, conteos_muertes, on=['Nombre_Departamento_Def', 'grupo_edad'], how='left')
    muertes_completas_df['count'] = muertes_completas_df['count'].fillna(0).astype(int)
    muertes_completas_df.rename(columns={'Nombre_Departamento_Def': 'DPNOM'}, inplace=True)
    muertes_completas_df.replace({'DPNOM': {'ARCHIPIÉLAGO DE SAN ANDRÉS Y PROVIDENCIA Y SANTA CATALINA': 'ARCHIPIÉLAGO DE SAN ANDRÉS'}}, inplace=True)

    # Procesamiento de Población (requiere 'poblacion' y 'cols_mujeres' globales)
    poblacion_df_filtrada = poblacion[(poblacion['ANIO'] >= año_inicio) & (poblacion['ANIO'] <= año_fin)].copy()
    
    for col in cols_mujeres:
        if col in poblacion_df_filtrada.columns:
            poblacion_df_filtrada[col] = pd.to_numeric(poblacion_df_filtrada[col].astype(str).str.replace(',', '', regex=False), errors='coerce')
    
    poblacion_df_filtrada['40-54 años'] = poblacion_df_filtrada[[f'Mujeres_{edad}' for edad in range(40, 55) if f'Mujeres_{edad}' in poblacion_df_filtrada.columns]].sum(axis=1)
    poblacion_df_filtrada['55-64 años'] = poblacion_df_filtrada[[f'Mujeres_{edad}' for edad in range(55, 65) if f'Mujeres_{edad}' in poblacion_df_filtrada.columns]].sum(axis=1)
    poblacion_df_filtrada['65-74 años'] = poblacion_df_filtrada[[f'Mujeres_{edad}' for edad in range(65, 75) if f'Mujeres_{edad}' in poblacion_df_filtrada.columns]].sum(axis=1)
    
    cols_edad_75_mas = [f'Mujeres_{edad}' for edad in range(75, 85) if f'Mujeres_{edad}' in poblacion_df_filtrada.columns]
    if 'Mujeres_85 y más' in poblacion_df_filtrada.columns:
        cols_edad_75_mas.append('Mujeres_85 y más')
    poblacion_df_filtrada['75+ años'] = poblacion_df_filtrada[cols_edad_75_mas].sum(axis=1)

    columnas_pob_interes = ['DPNOM'] + grupos_edad_estandar # DPNOM debe existir en poblacion_df_filtrada
    poblacion_seleccionada_df = poblacion_df_filtrada[columnas_pob_interes]

    poblacion_melted_df = poblacion_seleccionada_df.melt(id_vars=['DPNOM'], value_vars=grupos_edad_estandar,
                                                         var_name='grupo_edad', value_name='poblacion_sum_anios')
    
    poblacion_agrupada_df = poblacion_melted_df.groupby(['DPNOM', 'grupo_edad'], observed=False)['poblacion_sum_anios'].sum().reset_index()
    poblacion_agrupada_df.rename(columns={'poblacion_sum_anios': 'poblacion'}, inplace=True)
    # Asegurar consistencia en DPNOM para el merge
    poblacion_agrupada_df.replace({'DPNOM': {'ARCHIPIÉLAGO DE SAN ANDRÉS Y PROVIDENCIA Y SANTA CATALINA': 'ARCHIPIÉLAGO DE SAN ANDRÉS'}}, inplace=True)

    data_final_df = pd.merge(muertes_completas_df, poblacion_agrupada_df, on=['DPNOM', 'grupo_edad'], how='left')
    data_final_df['poblacion'] = data_final_df['poblacion'].fillna(0) 

    tabla_detalle = data_final_df[['DPNOM', 'grupo_edad', 'count', 'poblacion']].copy()

    proporcion_segi = {'40-54 años': 12,'55-64 años': 9,'65-74 años': 7,'75+ años': 4}
    total_p_segi = sum(proporcion_segi.values())
    proporcion_segi_normalizada = {k: v / total_p_segi for k, v in proporcion_segi.items()}

    data_final_df['proporcion_segi'] = data_final_df['grupo_edad'].map(proporcion_segi_normalizada).fillna(0)

    data_final_df['tasa_cruda_estrato'] = 0.0
    mask_poblacion_valida = data_final_df['poblacion'] > 0
    if mask_poblacion_valida.any(): # Solo calcular si hay alguna población válida
        data_final_df.loc[mask_poblacion_valida, 'tasa_cruda_estrato'] = \
            data_final_df.loc[mask_poblacion_valida, 'count'] / data_final_df.loc[mask_poblacion_valida, 'poblacion']
    
    data_final_df['tasa_ajustada_parcial'] = data_final_df['proporcion_segi'] * data_final_df['tasa_cruda_estrato']
    data_final_df['tasa_ajustada_parcial'] = pd.to_numeric(data_final_df['tasa_ajustada_parcial'], errors='coerce').fillna(0)

    tae_por_departamento_df = data_final_df.groupby('DPNOM', observed=False)['tasa_ajustada_parcial'].sum().reset_index(name='TAE')
    tae_por_departamento_df['TAE'] *= 100000

    mortalidad_agregada_df = data_final_df.groupby('DPNOM', observed=False).agg(
        FALLECIDAS=('count', 'sum'),
        TOTAL_MUJERES=('poblacion', 'sum')
    ).reset_index()
    
    mortalidad_agregada_df['Tasa_Mortalidad_Cruda'] = 0.0
    mask_pob_total_valida = mortalidad_agregada_df['TOTAL_MUJERES'] > 0
    if mask_pob_total_valida.any():
        mortalidad_agregada_df.loc[mask_pob_total_valida, 'Tasa_Mortalidad_Cruda'] = \
            (mortalidad_agregada_df.loc[mask_pob_total_valida, 'FALLECIDAS'] / mortalidad_agregada_df.loc[mask_pob_total_valida, 'TOTAL_MUJERES']) * 100000

    resumen_df = pd.merge(mortalidad_agregada_df, tae_por_departamento_df, on='DPNOM', how='left')
    
    return resumen_df, tabla_detalle

In [12]:
tasas, resumen = procesar_tasas_por_periodo(2009, 2023)

In [13]:
resumen

,DPNOM,grupo_edad,count,poblacion
0,ANTIOQUIA,40-54 años,1283,9466112
1,ANTIOQUIA,55-64 años,1620,4916940
2,ANTIOQUIA,65-74 años,1571,2863150
3,ANTIOQUIA,75+ años,1711,1765449
4,ARAUCA,40-54 años,46,312290
...,...,...,...,...
107,VALLE DEL CAUCA,75+ años,1461,1482090
108,GRUPO AMAZONA,40-54 años,21,311721
109,GRUPO AMAZONA,55-64 años,7,117144
110,GRUPO AMAZONA,65-74 años,4,57190


In [14]:
tasas

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad_Cruda,TAE
0,ANTIOQUIA,6185,19011651,32.532682,38.466253
1,ARAUCA,137,543874,25.189658,30.488575
2,ARCHIPIÉLAGO DE SAN ANDRÉS,30,189144,15.860931,26.424814
3,ATLÁNTICO,3122,6773319,46.092617,54.323739
4,BOLÍVAR,1588,5135499,30.922019,35.703283
5,BOYACÁ,748,3751470,19.938851,21.533421
6,CALDAS,1012,3393571,29.821094,33.117195
7,CAQUETÁ,203,852787,23.804303,28.849114
8,CASANARE,159,862417,18.436557,23.606235
9,CAUCA,631,3713294,16.992999,19.036951


## **SMR: Standard Mortality Rate**

$$
\text{SMR} = \frac{\text{Número observado de muertes}}{\text{Número esperado de muertes}} = \frac{y_i}{e_i}
$$

$$
e_i = R_T  \times N_i
$$

$$
N_i \text{:  tamaño de la población i }
$$

$$
R_T = \frac{casos \, totales}{Pob \, total}
$$

### **SMR Por departamento**

Moraga, P. (s.f.). Indirect standardization to calculate expected cases. Recuperado el 24 de abril de 2025, de https://www.paulamoraga.com/presentation-course/#/indirect-standardiz.-to-calculate-expected-cases

In [15]:
tasas.head()

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad_Cruda,TAE
0,ANTIOQUIA,6185,19011651,32.532682,38.466253
1,ARAUCA,137,543874,25.189658,30.488575
2,ARCHIPIÉLAGO DE SAN ANDRÉS,30,189144,15.860931,26.424814
3,ATLÁNTICO,3122,6773319,46.092617,54.323739
4,BOLÍVAR,1588,5135499,30.922019,35.703283


In [16]:
def calcular_SMR(tasas):
    muertes_totales_nal = tasas['FALLECIDAS'].sum()
    mujeres_totales_nal = tasas['TOTAL_MUJERES'].sum()
    tasa_nacional = muertes_totales_nal / mujeres_totales_nal
    tasas['MUERTES_ESPERADAS'] = tasa_nacional * tasas['TOTAL_MUJERES']
    tasas['SMR'] = tasas['FALLECIDAS'] / tasas['MUERTES_ESPERADAS'] * 100
    return tasa_nacional, tasas
    

In [17]:
tasa_nacional, tasas = calcular_SMR(tasas)

In [18]:
tasa_nacional

0.0003739005349635145

In [19]:
tasas

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad_Cruda,TAE,MUERTES_ESPERADAS,SMR
0,ANTIOQUIA,6185,19011651,32.532682,38.466253,7108.466479,87.008921
1,ARAUCA,137,543874,25.189658,30.488575,203.354780,67.369943
2,ARCHIPIÉLAGO DE SAN ANDRÉS,30,189144,15.860931,26.424814,70.721043,42.420189
3,ATLÁNTICO,3122,6773319,46.092617,54.323739,2532.547598,123.275077
4,BOLÍVAR,1588,5135499,30.922019,35.703283,1920.165823,82.701191
5,BOYACÁ,748,3751470,19.938851,21.533421,1402.676640,53.326617
6,CALDAS,1012,3393571,29.821094,33.117195,1268.858012,79.756757
7,CAQUETÁ,203,852787,23.804303,28.849114,318.857516,63.664800
8,CASANARE,159,862417,18.436557,23.606235,322.458178,49.308720
9,CAUCA,631,3713294,16.992999,19.036951,1388.402613,45.447912


In [20]:
tasas.to_excel('tasas_final.xlsx', index=False)

### **Interpretación del SMR**

- **SMR = 100**: Mortalidad observada igual a la esperada.
- **SMR > 100**: Más muertes de las esperadas (mayor riesgo).
- **SMR < 100**: Menos muertes de las esperadas (menor riesgo).
